In [5]:
import tensorflow as tf
from keras.layers import Dense, LSTM, Activation, Bidirectional, Dropout
from keras.models import Sequential
import keras.backend as K

In [ ]:
def reconstructionLoss(yTrue, yPred):
    return K.sum(K.log(yTrue) - K.log(yPred))

In [ ]:
def divergenceLoss()

In [ ]:
def CreateEncoder(input_num, output_num, hyperparams):
    #TODO Bidirectional LSTM.
    # TODO MAYBE: Add in more regularization or different than dropout?
    model = Sequential([
        Bidirectional(LSTM(32, activation='tanh', return_sequences=True, input_shape=(batch_size, time_steps, features))),
        Dropout(0.2),
        Bidirectional(LSTM(32, activation='tanh', dropout=0.2, return_sequences=False)),
        Dropout(0.5),
        Dense(output_num)
    ])
    
    
    
    model.compile()
    return model

In [ ]:
# k is num of domains.
# encoders is a list of encoders.
# decoders is list of decoders.
# samples is a K x N array of samples, where the first index is the domain,
# the second index is the # of the sample in that domain.

def trainAutoencoders(k, encoders, decoders, samples):
    N = samples.shape[0]
    
    for i in range(k):
        while(not isConverged(encoders[i], decoders[i])):
            p_Xi_samples = samples[i,:]
            p_Z_samples = projectZ(encoders[i](samples[]))
            p_Ni_samples = None # TODO Something!
            